##### Copyright 2018 The TensorFlow Authors.


In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 加载文本

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://tensorflow.google.cn/tutorials/load_data/text"><img src="https://tensorflow.google.cn/images/tf_logo_32px.png"> 在 TensorFlow.org 上查看</a>   </td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/zh-cn/tutorials/load_data/text.ipynb"><img src="https://tensorflow.google.cn/images/colab_logo_32px.png">在 Google Colab 中运行</a> </td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/zh-cn/tutorials/load_data/text.ipynb"><img src="https://tensorflow.google.cn/images/GitHub-Mark-32px.png">在 GitHub 上查看源代码</a> </td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/zh-cn/tutorials/load_data/text.ipynb"><img src="https://tensorflow.google.cn/images/download_logo_32px.png">下载笔记本</a> </td>
</table>

本教程演示了两种加载和预处理文本的方法。

- 首先，您将使用 Keras 效用函数和预处理层。这包括用于将数据转换为 `tf.data.Dataset` 的 `tf.keras.utils.text_dataset_from_directory` 和用于数据标准化、词例化和向量化的 `tf.keras.layers.TextVectorization`。如果您是 TensorFlow 新手，则应当从这些开始。
- 然后，您将使用 `tf.data.TextLineDataset` 等较低级别的效用函数来加载文本文件，并使用 [TensorFlow Text](https://tensorflow.google.cn/text) API（如 `text.UnicodeScriptTokenizer` 和 `text.case_fold_utf8`）来预处理数据以实现粒度更细的控制。

In [2]:
!pip install "tensorflow-text==2.8.*"
!pip install cleanlab==v2.2.0
!pip install scikeras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import collections
import pathlib

import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import utils
from tensorflow.keras.layers import TextVectorization

import tensorflow_datasets as tfds
import tensorflow_text as tf_text

import re 
import string 
import pandas as pd 
from sklearn.metrics import accuracy_score, log_loss 
from sklearn.model_selection import cross_val_predict  
from scikeras.wrappers import KerasClassifier 

### 下载并探索数据集

首先，使用 `tf.keras.utils.get_file` 下载 Stack Overflow 数据集，然后探索目录结构：

In [4]:
data_url = 'https://github.com/traaaariad/-/raw/main/txt3.tar.gz'

dataset_dir = utils.get_file(
    origin=data_url,
    untar=True,
    cache_dir='txt',
    cache_subdir='')

dataset_dir = pathlib.Path(dataset_dir).parent


In [5]:
list(dataset_dir.iterdir())

[PosixPath('/tmp/.keras/txt3.tar.gz'),
 PosixPath('/tmp/.keras/test'),
 PosixPath('/tmp/.keras/train')]

In [6]:
train_dir = dataset_dir/'train'
list(train_dir.iterdir())

[PosixPath('/tmp/.keras/train/close'),
 PosixPath('/tmp/.keras/train/unknown'),
 PosixPath('/tmp/.keras/train/open')]

`train/csharp`、`train/java`、`train/python` 和 `train/javascript` 目录包含许多文本文件，每个文件都是一个 Stack Overflow 问题。

打印示例文件并检查数据：

In [7]:
sample_file = train_dir/'open/1.txt'

with open(sample_file) as f:
  print(f.read())

Variable "CA" missing javadoc


### 加载数据集

接下来，您将从磁盘加载数据并将其准备成适合训练的格式。为此，您将使用 `tf.keras.utils.text_dataset_from_directory` 效用函数来创建带标签的 `tf.data.Dataset`。如果您是 `tf.data` 新手，它是用于构建输入流水线的强大工具集合。（要了解更多信息，请参阅 [tf.data：构建 TensorFlow 输入流水线](../../guide/data.ipynb)指南。）

`tf.keras.utils.text_dataset_from_directory` API 需要如下目录结构：

```
train/
...csharp/
......1.txt
......2.txt
...java/
......1.txt
......2.txt
...javascript/
......1.txt
......2.txt
...python/
......1.txt
......2.txt
```

运行机器学习实验时，最佳做法是将数据集拆成三份：[训练](https://developers.google.com/machine-learning/glossary#training_set)、[验证](https://developers.google.com/machine-learning/glossary#validation_set)和[测试](https://developers.google.com/machine-learning/glossary#test-set)。

Stack Overflow 数据集已经拆分为训练集和测试集，但缺少验证集。

通过使用 `tf.keras.utils.text_dataset_from_directory` 并将 `validation_split` 设置为 `0.2`（即 20%），使用训练数据的 80:20 拆分创建验证集：

In [8]:
batch_size = 32
seed = 42

raw_train_ds = utils.text_dataset_from_directory(
    train_dir,
    batch_size=batch_size,
    validation_split=0.2,
    subset='training',
    seed=seed)

Found 1947 files belonging to 3 classes.
Using 1558 files for training.


正如前面的单元输出所示，训练文件夹中有 8,000 个样本，您将使用其中的 80%（即 6,400 个）进行训练。稍后您将学习到，可以通过将 `tf.data.Dataset` 直接传递给 `Model.fit` 来训练模型。

首先，遍历数据集并打印出一些样本来感受一下数据。

In [9]:
for text_batch, label_batch in raw_train_ds.take(1):
  
  for i in range(10):
    print("Error: ", text_batch.numpy()[i])
    print("Label:", label_batch.numpy()[i])

Error:  b'Parameter docType should be final'
Label: 2
Error:  b'Parameter docType should be final'
Label: 2
Error:  b'Parameter docType should be final'
Label: 2
Error:  b'Parameter docType should be final'
Label: 2
Error:  b'Since class "GeneratedCriteria" is designed to be inheritable, add javadoc documentation for overridable non-null method "addCriterion"'
Label: 1
Error:  b'Line longer than 80 characters'
Label: 1
Error:  b'Since class "GeneratedCriteria" is designed to be inheritable, add javadoc documentation for overridable non-null method "addCriterion"'
Label: 1
Error:  b'Parameter docType should be final'
Label: 2
Error:  b'Parameter docType should be final'
Label: 2
Error:  b'Since class "GeneratedCriteria" is designed to be inheritable, add javadoc documentation for overridable non-null method "addCriterion"'
Label: 1


标签为 `0`、`1`、`2` 或 `3`。要查看其中哪些对应于哪个字符串标签，可以检查数据集上的 `class_names` 属性：


In [10]:
for i, label in enumerate(raw_train_ds.class_names):
  print("Label", i, "corresponds to", label)

Label 0 corresponds to close
Label 1 corresponds to open
Label 2 corresponds to unknown


接下来，您将使用 `tf.keras.utils.text_dataset_from_directory` 创建验证集和测试集。您将使用训练集中剩余的 1,600 条评论进行验证。

注：使用 `tf.keras.utils.text_dataset_from_directory` 的 `validation_split` 和 `subset` 参数时，请确保要么指定随机种子，要么传递 `shuffle=False`，这样验证拆分和训练拆分就不会重叠。

In [11]:
# Create a validation set.
raw_val_ds = utils.text_dataset_from_directory(
    train_dir,
    batch_size=batch_size,
    validation_split=0.2,
    subset='validation',
    seed=seed)

Found 1947 files belonging to 3 classes.
Using 389 files for validation.


In [12]:
test_dir = dataset_dir/'test'

# Create a test set.
raw_test_ds = utils.text_dataset_from_directory(
    test_dir,
    batch_size=batch_size)

Found 1947 files belonging to 3 classes.


### 准备用于训练的数据集

接下来，您将使用 `tf.keras.layers.TextVectorization` 层对数据进行标准化、词例化和向量化。

- *标准化*是指预处理文本，通常是移除标点符号或 HTML 元素以简化数据集。
- *词例化*是指将字符串拆分为词例（例如，通过按空格分割将一个句子拆分为各个单词）。
- *向量化*是指将词例转换为编号，以便将它们输入到神经网络中。

所有这些任务都可以通过这一层来完成。（您可以在 `tf.keras.layers.TextVectorization` API 文档中了解有关这些内容的更多信息。）

请注意：

- 默认标准化会将文本转换为小写并移除标点符号 (`standardize='lower_and_strip_punctuation'`)。
- 默认分词器会按空格分割 (`split='whitespace'`)。
- 默认向量化模式为 `'int'` (`output_mode='int'`)。这会输出整数索引（每个词例一个）。此模式可用于构建考虑词序的模型。您还可以使用其他模式（例如 `'binary'`）来构建[词袋](https://developers.google.com/machine-learning/glossary#bag-of-words)模型。

您将使用 `TextVectorization` 构建两个模型来详细了解标准化、词例化和向量化：

- 首先，您将使用 `'binary'` 向量化模式来构建词袋模型。
- 随后，您将使用具有 1D ConvNet 的 `'int'` 模式。

In [13]:
VOCAB_SIZE = 10000

binary_vectorize_layer = TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='binary')

对于 `'int'` 模式，除了最大词汇量之外，您还需要设置显式最大序列长度 (`MAX_SEQUENCE_LENGTH`)，这会导致层将序列精确地填充或截断为 `output_sequence_length` 值：

In [14]:
MAX_SEQUENCE_LENGTH = 250

int_vectorize_layer = TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=MAX_SEQUENCE_LENGTH)

新增：Cleanlab相关

接下来，调用 `TextVectorization.adapt` 以使预处理层的状态适合数据集。这会使模型构建字符串到整数的索引。

注：在调用 `TextVectorization.adapt` 时请务必仅使用您的训练数据（使用测试集会泄漏信息）。

In [15]:
# Make a text-only dataset (without labels), then call `TextVectorization.adapt`.
train_text = raw_train_ds.map(lambda text, labels: text)
binary_vectorize_layer.adapt(train_text)
int_vectorize_layer.adapt(train_text)

打印使用这些层预处理数据的结果：

In [16]:
def binary_vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return binary_vectorize_layer(text), label

In [17]:
def int_vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return int_vectorize_layer(text), label

In [18]:
# Retrieve a batch (of 32 reviews and labels) from the dataset.
text_batch, label_batch = next(iter(raw_train_ds))
first_question, first_label = text_batch[0], label_batch[0]
print("Error", first_question)
print("Label", first_label)

Error tf.Tensor(b'Since class "GeneratedCriteria" is designed to be inheritable, add javadoc documentation for overridable non-null method "addCriterion"', shape=(), dtype=string)
Label tf.Tensor(1, shape=(), dtype=int32)


In [19]:
print("'binary' vectorized question:",
      binary_vectorize_text(first_question, first_label)[0])

'binary' vectorized question: tf.Tensor(
[[0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]], shape=(1, 73), dtype=float32)


In [20]:
print("'int' vectorized question:",
      int_vectorize_text(first_question, first_label)[0])

'int' vectorized question: tf.Tensor(
[[ 7 15 11  4 14  6  2 10 17  3 13 12  8  9  5 16  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0]], shape=(1, 250), dtype=int64)


如上所示，`TextVectorization` 的 `'binary'` 模式返回一个数组，表示哪些词例在输入中至少存在一次，而 `'int'` 模式将每个词例替换为一个整数，从而保留它们的顺序。

您可以通过在层上调用 `TextVectorization.get_vocabulary` 来查找每个整数对应的词例（字符串）：

In [21]:
#print("1289 ---> ", int_vectorize_layer.get_vocabulary()[1289])
#print("313 ---> ", int_vectorize_layer.get_vocabulary()[313])
#print("Vocabulary size: {}".format(len(int_vectorize_layer.get_vocabulary())))

差不多可以训练您的模型了。

作为最后的预处理步骤，将之前创建的 `TextVectorization` 层应用于训练集、验证集和测试集：

In [22]:
binary_train_ds = raw_train_ds.map(binary_vectorize_text)
binary_val_ds = raw_val_ds.map(binary_vectorize_text)
binary_test_ds = raw_test_ds.map(binary_vectorize_text)

int_train_ds = raw_train_ds.map(int_vectorize_text)
int_val_ds = raw_val_ds.map(int_vectorize_text)
int_test_ds = raw_test_ds.map(int_vectorize_text)

### 配置数据集以提高性能

以下是加载数据时应该使用的两种重要方法，以确保 I/O 不会阻塞。

- 从磁盘加载后，`Dataset.cache` 会将数据保存在内存中。这将确保数据集在训练模型时不会成为瓶颈。如果您的数据集太大而无法放入内存，也可以使用此方法创建高性能的磁盘缓存，这比许多小文件的读取效率更高。
- `Dataset.prefetch` 会在训练时将数据预处理和模型执行重叠。

您可以在[使用 tf.data API 提升性能](../../guide/data_performance.ipynb)指南的*预提取*部分中详细了解这两种方法，以及如何将数据缓存到磁盘。

In [23]:
AUTOTUNE = tf.data.AUTOTUNE

def configure_dataset(dataset):
  return dataset.cache().prefetch(buffer_size=AUTOTUNE)

In [24]:
binary_train_ds = configure_dataset(binary_train_ds)
binary_val_ds = configure_dataset(binary_val_ds)
binary_test_ds = configure_dataset(binary_test_ds)

int_train_ds = configure_dataset(int_train_ds)
int_val_ds = configure_dataset(int_val_ds)
int_test_ds = configure_dataset(int_test_ds)

### 训练模型。

是时候创建您的神经网络了。

对于 `'binary'` 向量化数据，定义一个简单的词袋线性模型，然后对其进行配置和训练：

在这里，我们构建了一个简单的神经网络，用于使用TensorFlow和Keras进行分类。

In [25]:
def get_net():
    net = tf.keras.Sequential(
        [
            layers.Dense(4),
        ]
    )  # outputs probability that text belongs to class 1

    net.compile(
        optimizer="adam",
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=tf.keras.metrics.CategoricalAccuracy(),
    )
    return net

In [26]:
model = KerasClassifier(get_net(), epochs=10)

In [27]:
num_crossval_folds = 3  # for efficiency; values like 5 or 10 will generally work better
pred_probs = cross_val_predict(
    model,
    text_batch,
    label_batch,
    cv=num_crossval_folds,
    method="predict_proba",
)

ValueError: ignored

In [ ]:
from cleanlab.filter import find_label_issues

ranked_label_issues = find_label_issues(
    labels=label_batch, pred_probs=pred_probs, return_indices_ranked_by="self_confidence"
)

In [ ]:
print(
    f"cleanlab found {len(ranked_label_issues)} potential label errors.\n"
    f"Here are indices of the top 1 most likely errors: \n {ranked_label_issues[:1]}"
)

接下来，您将使用 `'int'` 向量化层来构建 1D ConvNet：

In [ ]:
def create_model(vocab_size, num_labels):
  model = tf.keras.Sequential([
      layers.Embedding(vocab_size, 64, mask_zero=True),
      layers.Conv1D(64, 5, padding="valid", activation="relu", strides=2),
      layers.GlobalMaxPooling1D(),
      layers.Dense(num_labels)
  ])
  return model

In [ ]:
# `vocab_size` is `VOCAB_SIZE + 1` since `0` is used additionally for padding.
int_model = create_model(vocab_size=VOCAB_SIZE + 1, num_labels=4)
int_model.compile(
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])
history = int_model.fit(int_train_ds, validation_data=int_val_ds, epochs=1)

比较两个模型：

In [ ]:
print("Linear model on binary vectorized data:")
print(binary_model.summary())

In [ ]:
print("ConvNet model on int vectorized data:")
print(int_model.summary())

在测试数据上评估两个模型：

In [ ]:
binary_loss, binary_accuracy = binary_model.evaluate(binary_test_ds)
int_loss, int_accuracy = int_model.evaluate(int_test_ds)

print("Binary model accuracy: {:2.2%}".format(binary_accuracy))
print("Int model accuracy: {:2.2%}".format(int_accuracy))

注：此示例数据集代表了一个相当简单的分类问题。更复杂的数据集和问题会在预处理策略和模型架构上带来微妙但显著的差异。务必尝试不同的超参数和周期来比较各种方法。

### 导出模型

在上面的代码中，您在向模型馈送文本之前对数据集应用了 `tf.keras.layers.TextVectorization`。如果您想让模型能够处理原始字符串（例如，为了简化部署），您可以在模型中包含 `TextVectorization` 层。

为此，您可以使用刚刚训练的权重创建一个新模型：

In [ ]:
export_model = tf.keras.Sequential(
    [binary_vectorize_layer, binary_model,
     layers.Activation('sigmoid')])

export_model.compile(
    loss=losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer='adam',
    metrics=['accuracy'])

# Test it with `raw_test_ds`, which yields raw strings
loss, accuracy = export_model.evaluate(raw_test_ds)
print("Accuracy: {:2.2%}".format(binary_accuracy))

现在，您的模型可以将原始字符串作为输入，并使用 `Model.predict` 预测每个标签的得分。定义一个函数来查找得分最高的标签：

In [ ]:
def get_string_labels(predicted_scores_batch):
  predicted_int_labels = tf.math.argmax(predicted_scores_batch, axis=1)
  predicted_labels = tf.gather(raw_train_ds.class_names, predicted_int_labels)
  return predicted_labels